In [ ]:
# Core Libraries
import numpy as np
import itertools

# Keras Libraries for Model Building
from keras.models import Sequential, load_model
from keras.layers import Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard


In [ ]:
ROWS = 256
COLS = 256
CHANNELS = 3

In [ ]:
import os
import shutil
import random

def split_dataset(main_folder, output_folder, split_ratio=0.5):
    train_folder = os.path.join(output_folder, 'train')
    test_folder = os.path.join(output_folder, 'test')

    # Create train and test directories
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)

    # Iterate through each class (subfolder)
    for class_name in os.listdir(main_folder):
        class_path = os.path.join(main_folder, class_name)

        if not os.path.isdir(class_path):  # Ignore non-folder files
            continue

        # Create class folders in train and test directories
        train_class_folder = os.path.join(train_folder, class_name)
        test_class_folder = os.path.join(test_folder, class_name)
        os.makedirs(train_class_folder, exist_ok=True)
        os.makedirs(test_class_folder, exist_ok=True)

        # Get all image files
        images = [img for img in os.listdir(class_path) if img.lower().endswith(('jpg', 'jpeg', 'png'))]
        random.shuffle(images)

        # Split images
        split_index = int(len(images) * split_ratio)
        train_images = images[:split_index]
        test_images = images[split_index:]

        # Move images to respective folders
        for img in train_images:
            shutil.copy(os.path.join(class_path, img), os.path.join(train_class_folder, img))

        for img in test_images:
            shutil.copy(os.path.join(class_path, img), os.path.join(test_class_folder, img))

        print(f"Class '{class_name}': {len(train_images)} in train, {len(test_images)} in test.")

    print("Dataset split completed successfully!")

# Example usage:
main_folder = "CUB_200_2011/images"  # Replace with your dataset path
output_folder = "CUB_200_2011/"  # Replace with your desired output path
split_dataset(main_folder, output_folder, split_ratio=0.5)

In [ ]:
train_image_generator = ImageDataGenerator(horizontal_flip=True, rescale=1./255, rotation_range=45) 
test_image_generator = ImageDataGenerator(horizontal_flip=False, rescale=1./255, rotation_range=0)

train_generator = train_image_generator.flow_from_directory('CUB_200_2011/train', target_size=(ROWS, COLS), class_mode='categorical')
test_generator = test_image_generator.flow_from_directory('CUB_200_2011/test', target_size=(ROWS, COLS), class_mode='categorical') 

In [ ]:
train_generator.reset() 
test_generator.reset()

model = Sequential() 
model.add(Conv2D (64, (3,3), input_shape=(ROWS, COLS, CHANNELS))) 
model.add(Activation('relu')) 
model.add(Conv2D(64, (3,3))) 
model.add(Activation("relu")) 
model.add(MaxPooling2D(pool_size=(4,4))) 
model.add(Conv2D(64, (3,3))) 
model.add(Activation("relu")) 
model.add(Conv2D(64, (3,3))) 
model.add(Activation ('relu')) 
model.add(MaxPooling2D(pool_size=(4,4))) 
model.add(Flatten()) 
model.add(Dropout(0.5)) 
model.add(Dense(400)) 
model.add(Activation('relu')) 
model.add (Dropout(0.5)) 
model.add(Dense(200)) 
model.add(Activation('softmax')) 

model.compile(loss='categorical_crossentropy', optimizer='adamax', metrics=['accuracy'])

model.summary()

In [ ]:
from keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir='./logs/custom')

model.fit(train_generator, steps_per_epoch=512, callbacks=[tensorboard], verbose=2)

In [ ]:
print(model.evaluate(test_generator, steps=1000))